In [1]:
import os
from datetime import datetime
import IPython
import IPython.display
#matplotlib seaborn
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
#ray modin.pandas
import numpy as np
import ray
import modin.pandas as pd
ray.init(runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}})
#sklearn
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
#tensorflow
import tensorflow as tf

2023-03-02 18:28:51,509	INFO worker.py:1509 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
(pid=) c:\Users\ZIYAD\anaconda3\envs\nlp\lib\site-packages\ray\dashboard\modules\reporter\reporter_agent.py:46: UserWarning: `gpustat` package is not installed. GPU monitoring is not available. To have full functionality of the dashboard please install `pip install ray[default]`.)
(pid=)   warnings.warn(


In [4]:
df=pd.read_csv(r"C:\Users\ZIYAD\Desktop\IOT project\TRRR\trafficData02112014.csv")
df

,status,avgMeasuredTime,avgSpeed,extID,medianMeasuredTime,TIMESTAMP,vehicleCount,_id,REPORT_ID
0,OK,66,56,668,66,2014-02-13T11:30:00,7,190000,158324
1,OK,69,53,668,69,2014-02-13T11:35:00,5,190449,158324
2,OK,69,53,668,69,2014-02-13T11:40:00,6,190898,158324
3,OK,70,52,668,70,2014-02-13T11:45:00,3,191347,158324
4,OK,64,57,668,64,2014-02-13T11:50:00,6,191796,158324
...,...,...,...,...,...,...,...,...,...
25097088,OK,1798,2,623,1798,2014-11-13T10:20:00,0,32507360,210199
25097089,OK,1798,2,623,1798,2014-11-13T10:30:00,0,32507801,210199
25097090,OK,1798,2,623,1798,2014-11-13T10:35:00,0,32508244,210199
25097091,OK,1798,2,623,1798,2014-11-13T10:40:00,0,32508648,210199


In [3]:
df

,status,avgMeasuredTime,avgSpeed,extID,medianMeasuredTime,TIMESTAMP,vehicleCount,_id,REPORT_ID
0,OK,66,56,668,66,2014-02-13T11:30:00,7,190000,158324
1,OK,69,53,668,69,2014-02-13T11:35:00,5,190449,158324
2,OK,69,53,668,69,2014-02-13T11:40:00,6,190898,158324
3,OK,70,52,668,70,2014-02-13T11:45:00,3,191347,158324
4,OK,64,57,668,64,2014-02-13T11:50:00,6,191796,158324
...,...,...,...,...,...,...,...,...,...
25097088,OK,1798,2,623,1798,2014-11-13T10:20:00,0,32507360,210199
25097089,OK,1798,2,623,1798,2014-11-13T10:30:00,0,32507801,210199
25097090,OK,1798,2,623,1798,2014-11-13T10:35:00,0,32508244,210199
25097091,OK,1798,2,623,1798,2014-11-13T10:40:00,0,32508648,210199


In [47]:
#df=pd.read_csv("183063.csv")
#date_time = pd.to_datetime(df.pop('TIMESTAMP'), format='%Y-%m-%dT%H:%M:%S')
df.drop(columns=['extID','REPORT_ID','_id','Unnamed: 0','Min','status','medianMeasuredTime'],inplace=True)
df.head()

,avgMeasuredTime,avgSpeed,TIMESTAMP,vehicleCount,M,D,H
0,86,48,2014-02-13T11:30:00,4,2,13,11
1,91,45,2014-02-13T11:35:00,3,2,13,11
2,105,39,2014-02-13T11:40:00,7,2,13,11
3,112,36,2014-02-13T11:45:00,10,2,13,11
4,97,42,2014-02-13T11:50:00,8,2,13,11


In [48]:
df3 = pd.DataFrame(columns=df.columns)
#from 5min to 1hour
for m in df['M'].unique():
    for d in df['D'].unique():
        for h in df['H'].unique():
            DDD=df[(df['H'] == h)&(df['D'] == d)&(df['M'] == m)]
            if DDD['avgSpeed'].mean() > 0:
                #print([[DDD['avgMeasuredTime'].mean(),DDD['avgSpeed'].mean(),DDD['vehicleCount'].sum(),DDD['M'].mean(),DDD['D'].mean(),DDD['H'].mean()]])
                df4=pd.DataFrame([[DDD['avgMeasuredTime'].mean(),DDD['avgSpeed'].mean(),DDD['TIMESTAMP'].values[0],
                                   DDD['vehicleCount'].sum(),DDD['M'].mean(),DDD['D'].mean(),DDD['H'].mean()]],
                                     columns=df.columns.tolist())
                df3=pd.concat([df3,df4],ignore_index=True)

#extract datetime from timestamp
date_time = pd.to_datetime(df3.pop('TIMESTAMP'), format='%Y-%m-%dT%H:%M:%S')

In [49]:
df=df3
df['vehicleCount']=df['vehicleCount'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5028 entries, 0 to 5027
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   avgMeasuredTime  5028 non-null   float64
 1   avgSpeed         5028 non-null   float64
 2   vehicleCount     5028 non-null   int32  
 3   M                5028 non-null   float64
 4   D                5028 non-null   float64
 5   H                5028 non-null   float64
dtypes: float64(5), int32(1)
memory usage: 216.2 KB


In [23]:
df['avgSpeed']=df['avgSpeed'].astype(int)

In [50]:
df.drop(columns=['M','D','H'],inplace=True)
df.head()

,avgMeasuredTime,avgSpeed,vehicleCount
0,94.333333,44.166667,37
1,104.750000,39.666667,99
2,111.333333,37.416667,136
3,128.500000,32.083333,150
4,107.250000,39.250000,125


In [51]:
mean = df.mean()
std = df.std()
df = (df - mean) / std
df.head()

,avgMeasuredTime,avgSpeed,vehicleCount
0,-0.438201,0.164485,-0.208904
1,0.153914,-0.389065,1.353849
2,0.528130,-0.665840,2.286459
3,1.503935,-1.321900,2.639339
4,0.296021,-0.440320,2.009197


In [52]:
timestamp_s = date_time.map(pd.Timestamp.timestamp)
day = 24*60*60
year = (365.2425)*day

df['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
df['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
df['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
df['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))
#df['Weekday'] = [datetime.weekday(date) for date in date_time]
#df["IsWeekend"] = df["Weekday"] >= 5

df.head()
timestamp_s

0       1.392291e+09
1       1.392293e+09
2       1.392296e+09
3       1.392300e+09
4       1.392304e+09
            ...     
5023    1.415772e+09
5024    1.415776e+09
5025    1.415779e+09
5026    1.415783e+09
5027    1.415786e+09
Name: TIMESTAMP, Length: 5028, dtype: float64

In [53]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

from pandas.plotting import scatter_matrix
from sklearn.neighbors import KNeighborsRegressor
from sklearn import linear_model

In [54]:
y1=df.avgSpeed[1:]
df1=df.iloc[:-1]
x_train, x_test, y_train, y_test = train_test_split(df1, y1, test_size = 0.20,random_state=21)

In [61]:
scoring = 'accuracy'
models= []
models.append(('LR', LinearRegression()))
models.append(('SVM', SVR()))
models.append(('KNN', KNeighborsRegressor(n_neighbors=48)))
results= names= tmodels= []
for name, model in models:
    names.append(name)
    msg = "For %s Model" % (name)
    print(msg)
    clf = model
    clf.fit(x_train, y_train)
    predict = clf.predict(x_test)
    accuracy = clf.score(x_test, y_test)
    print("Test Accuracy:",accuracy)
    tmodels.append(clf)
    print("*******************************************************************")

For LR Model
Test Accuracy: 0.41672933108977694
*******************************************************************
For SVM Model
Test Accuracy: 0.4655619170729991
*******************************************************************
For KNN Model
Test Accuracy: 0.4399653241230149
*******************************************************************


In [ ]:






inputs, labels = self.example
plt.figure(figsize=(12, 8))
plot_col_index = self.column_indices[plot_col]
max_n = min(max_subplots, len(inputs))

ppre=[]
llabels=[]

for n in range(max_n):
  print("n=",n)
  plt.subplot(max_n, 1, n+1)
  plt.ylabel(f'{plot_col} [normed]')
  plt.plot(self.input_indices, inputs[n, :, plot_col_index],
           label='Inputs', marker='.', zorder=-10)

  if self.label_columns:
    label_col_index = self.label_columns_indices.get(plot_col, None)
  else:
    label_col_index = plot_col_index

  if label_col_index is None:
    continue
    
  llabels.append(labels[n, :, label_col_index])
  plt.scatter(self.label_indices, labels[n, :, label_col_index],
              edgecolors='k', label='Labels', c='#2ca02c', s=64)
  if model is not None:
    predictions = model(inputs)
    ppre.append(predictions[n, :, label_col_index])
    plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                marker='X', edgecolors='k', label='Predictions',
                c='#ff7f0e', s=64)

  if n == 0:
    plt.legend()

plt.xlabel('Time [H]')